In [3]:
# Se importan las librerias a utilizar
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel

In [4]:
# se carga el dataset
df  = pd.read_csv('./dataset/movies_limpio.csv', delimiter=',')
# se selecciona las columnas de interés
columnas_seleccionadas = ['overview', 'popularity', 'tagline', 'title']  # Reemplaza con los nombres de las columnas que deseas seleccionar
df = df[columnas_seleccionadas]
df.sample(10)

C:\Users\Area-Desa15\AppData\Local\Temp\ipykernel_17372\2947862856.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df  = pd.read_csv('./dataset/movies_limpio.csv', delimiter=',')


,overview,popularity,tagline,title
28408,Lisa and Marine live a life of relative compan...,1.877117,NaN,The Adopted
33367,A road trip from Karachi to Lahore where 5 fri...,0.087996,NaN,Karachi se Lahore
3153,Attractive Manhattanite Allison Jones has it a...,7.246119,Living with a roommate can be murder.,Single White Female
28490,A look into the judicial scandal that rocked t...,1.802009,NaN,Kids for Cash
16227,When a young Iranian's mother doesn't meet her...,0.578872,NaN,The Mirror
44217,A bumbling Paris policeman is doggedly determi...,4.620808,What if Paris was yours?,2 Alone in Paris
2073,"Unable, due to the seal of the confessional, t...",10.424982,Crushed lips don’t talk.,I Confess
26876,Jeong Yoon is a caring wife and mother and a s...,2.329351,NaN,Way Back Home
7096,A small-town girl wins a date with a Hollywood...,5.924226,"In every love story, there's only room for one...",Win a Date with Tad Hamilton!
2172,Scientists in the Antarctic are confronted by ...,16.83125,Man is The Warmest Place to Hide.,The Thing


In [5]:
# Se eliminan los registros duplicados
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

In [6]:
#se imputan los nulos o nan con ''
titulo = df['title'].fillna('')
resumen = df['overview'].fillna('')
frase   = df['tagline'].fillna('')

In [7]:
tfidf = TfidfVectorizer(stop_words='english', lowercase=True) # configuracion de TfidfVectorizery, en idioma ingles

In [12]:
# se crea la una matriz TF-IDF con las puntaciones del campo title
tfidf_matrix = tfidf.fit_transform(titulo)

In [13]:
print(tfidf_matrix)

  (0, 19260)	0.5339365409301036
  (0, 20542)	0.8455245533156301
  (1, 10646)	1.0
  (2, 12911)	0.4385911829358217
  (2, 14434)	0.4950577534438312
  (2, 8663)	0.7500370624216536
  (3, 6866)	0.7982840579051067
  (3, 21743)	0.6022811327731894
  (4, 9820)	0.506957062721474
  (4, 2927)	0.6143944491204045
  (4, 7111)	0.6045775363399223
  (5, 9141)	1.0
  (6, 17314)	1.0
  (7, 9632)	0.8034553065631692
  (7, 20410)	0.5953650731740011
  (8, 5290)	0.5383829329153123
  (8, 19437)	0.8427003130090235
  (9, 8404)	1.0
  (10, 15835)	0.8041581820460744
  (10, 886)	0.5944153583550419
  (11, 12048)	0.6667325388456612
  (11, 5265)	0.4474140827223685
  (11, 6047)	0.5960607017964868
  (12, 1784)	1.0
  (13, 14129)	1.0
  :	:
  (45352, 12669)	0.47019824252249104
  (45353, 5360)	0.6886920741841239
  (45353, 9136)	0.7250539476176715
  (45354, 13331)	1.0
  (45355, 13480)	1.0
  (45356, 17095)	0.6666752269608563
  (45356, 13024)	0.5325537889656329
  (45356, 19034)	0.5214696574250882
  (45357, 9553)	0.8374075465717441


In [14]:
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix) # se calcula la matriz de coseno para los vectores.

In [15]:
def get_recommendations(df, column, value, cosine_similarities, limit=5):
    """ Retorna un dataframe de 5 registros recomendados según la 
        Frecuencia de término Frecuencia de documento inversa (TF-IDF) 
        y la similitud del coseno.
    
    Args:
        df (object): dataframe que contiene el texto a analizar. 
        column (string): nombre de la columna utilizada. 
        value (string): nombre del titulo a tener las recomendaciones.
        cosine_similarities (array): matriz de similitud del cosenol
        limit (int, optional): retorna la cantidad de registros a retornar recomendados. 
        
    Returns: 
        Pandas dataframe. 
    """
    
    # Return indices for the target dataframe column and drop any duplicates
    indices = pd.Series(df.index, index=df[column]).drop_duplicates()

    # Get the index for the target value
    target_index = indices[value]

    # Get the cosine similarity scores for the target value
    cosine_similarity_scores = list(enumerate(cosine_similarities[target_index]))

    # Sort the cosine similarities in order of closest similarity
    cosine_similarity_scores = sorted(cosine_similarity_scores, key=lambda x: x[1], reverse=True)

    # Return tuple of the requested closest scores excluding the target item and index
    cosine_similarity_scores = cosine_similarity_scores[1:limit+1]

    # Extract the tuple values
    index = (x[0] for x in cosine_similarity_scores)
    scores = (x[1] for x in cosine_similarity_scores)    

    # Get the indices for the closest items
    recommendation_indices = [i[0] for i in cosine_similarity_scores]

    # Get the actutal recommendations
    recommendations = df[column].iloc[recommendation_indices]

    # Return a dataframe
    df = pd.DataFrame(list(zip(index, recommendations, scores)), 
                      columns=['index','recommendation', 'cosine_similarity_score']) 

    return df

In [16]:
recommendations = get_recommendations(df, 
                                      'title', 
                                      'Toy Story', 
                                      cosine_similarities)
recommendations.head(5)

,index,recommendation,cosine_similarity_score
0,2994,Toy Story 2,1.000000
1,15337,Toy Story 3,1.000000
2,4794,The Toy,0.845525
3,16142,The Toy,0.845525
4,21898,Toy Story of Terror!,0.817672


In [17]:
def obtenerListaPeliculas(pelicula):
    recommendations = get_recommendations(df, 
                                      'title', 
                                      pelicula, 
                                      cosine_similarities)
    return recommendations['recommendation']
    